In [61]:
import cv2
import numpy as np
import math

In [2]:
path1 = r'imgs/IMG_20240510_172748.jpg'
path2 = r'imgs/IMG_20240510_172837.jpg'
path3 = r'imgs/IMG_20240510_172930.jpg'

In [102]:
def getCountorsB(frame_orig, gray = False, draw = True):
    # if not gray:
    #     frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # else:
    #     frame_gray = frame
    frame = frame_orig.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    
    print(len(cnts))
    # cnt = [c[0] for c in cnts[:3]]
    cnt = [c[0] for c in cnts[:1]]
    # cnt = [c for c in cnts[:3]]
    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # Draw the center of the contour
        # cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
        # contours = [c[0] for c in cnt]

        cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)

        ellipse = cv2.fitEllipse(cnt[0])

        (cx,cy), (d1,d2), angle = ellipse
        print(int(cx), int(cy))
        # cv2.circle(frame_orig, (int(cx), int(cy)), 5, (255, 255, 255), -1)
        # cv2.imshow('center', frame_orig)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
                

        ellipse_img = frame_orig.copy()
        cv2.ellipse(ellipse_img, (int(cx),int(cy)), (int(d1/2),int(d2/2)), angle, 0, 360, (0,0,255), 4)

        if angle > 90:
            angle = angle - 90
        else:
            angle = angle + 90
        print('center: (', cx,cy, ')', 'diameters: (', d1, d2, ')', 'angle:', angle)

        r1 = min(d1,d2)/2
        r2 = max(d1,d2)/2

        x1 = np.intp( cx + math.cos(math.radians(angle))*r2 )
        y1 = np.intp( cy + math.sin(math.radians(angle))*r2 )
        x2 = np.intp( cx + math.cos(math.radians(angle+180))*r2 )
        y2 = np.intp( cy + math.sin(math.radians(angle+180))*r2 )
        x3 = np.intp( cx + math.cos(math.radians(angle+90))*r1 )
        y3 = np.intp( cy + math.sin(math.radians(angle+90))*r1 )
        x4 = np.intp( cx + math.cos(math.radians(angle+270))*r1 )
        y4 = np.intp( cy + math.sin(math.radians(angle+270))*r1 )

        input_pts = np.float32([[x1,y1], [x2,y2], [x3,y3], [x4,y4]])

        # output points from circle of radius = max semi-major radii
        ox1 = np.intp( cx + math.cos(math.radians(angle))*r2 )
        oy1 = np.intp( cy + math.sin(math.radians(angle))*r2 )
        ox2 = np.intp( cx + math.cos(math.radians(angle+180))*r2 )
        oy2 = np.intp( cy + math.sin(math.radians(angle+180))*r2 )
        ox3 = np.intp( cx + math.cos(math.radians(angle+90))*r2 )
        oy3 = np.intp( cy + math.sin(math.radians(angle+90))*r2 )
        ox4 = np.intp( cx + math.cos(math.radians(angle+270))*r2 )
        oy4 = np.intp( cy + math.sin(math.radians(angle+270))*r2 )

        cx = np.intp(cx)
        cy = np.intp(cy)
        r2 = np.intp(r2)
        print(int(cx), int(cy))
        # cv2.circle(frame_orig, (int(cx), int(cy)), 5, (255, 0, 255), -1)
        # cv2.imshow('center', frame_orig)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
#         ellipse_pts = ellipse.copy()

# # draw white circle on copy of ellipse
#         cv2.circle(ellipse_pts, (cx,cy), r2, (255,255,255), 4)

        output_pts = np.float32([[ox1,oy1], [ox2,oy2], [ox3,oy3], [ox4,oy4]])

        h = cv2.getPerspectiveTransform(input_pts, output_pts)

        # warp image
        result = cv2.warpPerspective(frame, h, (frame.shape[1],frame.shape[0]))

        # cv2.ellipse(frame, ellipse,(0,255,0), 10)
        # cv2.circle(frame, (int(cx), int(cy)), 7, (0, 255, 255), -1)
        # for i in range(len(cnt)):
        #     print(i)
        #     x, y, w, h = cnt[i][4]
        #     frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)



    cnt = cnt[0]
    l = tuple(cnt[cnt[:,:,0].argmin()][0])
    r = tuple(cnt[cnt[:,:,0].argmax()][0])
    t = tuple(cnt[cnt[:,:,1].argmin()][0])
    b = tuple(cnt[cnt[:,:,1].argmax()][0])
    print(cv2.contourArea(cnt[0]) / float(frame.shape[0] * frame.shape[1]))

    return result, ellipse_img, cnt, (l, r, t, b), (d1/2, d2/2)

In [103]:
img = cv2.imread(path2)

w, h = img.shape[1]//3, img.shape[0]//3

img = cv2.resize(img, (w, h))

result, ellipse_img,  cnts1, extreme_pts, radiuses = getCountorsB(img)

cv2.imshow('Orig', img)
cv2.imshow('Ellipse', ellipse_img)
cv2.imshow('Circle', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

232
844 547
center: ( 844.4893798828125 547.0427856445312 ) diameters: ( 782.6536254882812 796.6148681640625 ) angle: 164.00758361816406
844 547
0.0


-1

In [99]:
# getCountorsB(img)
getCountorsB(result)

228
center: ( 844.4900512695312 547.0548095703125 ) diameters: ( 788.8019409179688 790.4871215820312 ) angle: 173.43698120117188
0.0


(array([[[221, 221, 221],
         [220, 220, 220],
         [220, 220, 220],
         ...,
         [216, 216, 216],
         [ 74,  74,  74],
         [  0,   0,   0]],
 
        [[220, 220, 220],
         [221, 221, 221],
         [221, 221, 221],
         ...,
         [218, 218, 218],
         [ 75,  75,  75],
         [  0,   0,   0]],
 
        [[220, 220, 220],
         [221, 221, 221],
         [220, 220, 220],
         ...,
         [215, 215, 215],
         [ 74,  74,  74],
         [  0,   0,   0]],
 
        ...,
 
        [[  0,   0,   0],
         [ 50,  50,  50],
         [221, 221, 221],
         ...,
         [200, 200, 200],
         [200, 200, 200],
         [199, 199, 199]],
 
        [[  0,   0,   0],
         [ 48,  48,  48],
         [221, 221, 221],
         ...,
         [200, 200, 200],
         [200, 200, 200],
         [198, 198, 198]],
 
        [[  0,   0,   0],
         [ 49,  49,  49],
         [222, 222, 222],
         ...,
         [200, 200, 200],
  

In [56]:
extreme_pts

((365, 598), (1293, 605), (832, 151), (798, 1059))

In [60]:
rx, ry = radiuses

dist = ry - rx

(453.0917053222656, 464.26202392578125)

In [142]:
cv2.circle(img_cnts, l, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, r, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, t, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, b, 8, (0, 255, 0), -1)

p1 = (l[0], t[1])
p2 = (r[0], t[1])
p3 = (l[0], b[1])
p4 = (r[0], b[1])

# p_c = (int((l[0] + r[0] + t[0] + b[0])/4), int((l[1] + r[1] + t[1] + b[1])/4))

# cv2.circle(img_cnts, p_c, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p1, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p2, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p3, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p4, 8, (255, 255, 0), -1)

# Display the image
cv2.imshow('Extreme Points', img_cnts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [143]:
pts1 = np.float32([l, t, b, r])
# pts2 = np.float32([p1, p2, p3, p4])
pts2 = np.float32([p1, p2, p3, p4])

M = cv2.getPerspectiveTransform(pts1, pts2)

result = cv2.warpPerspective(img, M, (w, h))

# cv2.circle(result, (450, 450), 8, (255, 255, 0), -1)

cv2.imshow('TR', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [145]:
cv2.imwrite('result.jpg', result)

True

In [84]:
# img = cv2.imread(path2)

# w, h = img.shape[1]//4, img.shape[0]//4

img = result.copy()

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# gray_blurred = cv2.medianBlur(gray_img, 21, 1)
gray_blurred = cv2.bilateralFilter(gray_img, 15, 75, 75)
# gray_blurred = cv2.GaussianBlur(gray_img, (21, 21), 7)

cv2.imshow('Detected Circles', gray_blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20,  
                                   param1=100, param2=250, minRadius=0, maxRadius=0)

# detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
#                                    param1=50, param2=220, minRadius=0, maxRadius=0)

if detected_circles is not None:
    detected_circles = np.uint16(np.around(detected_circles))
    circles = detected_circles[0, :]
    circles = sorted(circles, key=lambda x: x[2], reverse=True)
    
    # Draw only the 5 largest circles
    i = 0
    for circle in circles:
        (x, y, r) = circle
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 1, (0, 0, 255), 3)
        if i == 5: break


cv2.imshow('Detected Circles', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
def detect_black_white_yellow(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define thresholds for black, white, and yellow colors in HSV space
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 100])
    lower_white = np.array([0, 0, 175])
    upper_white = np.array([180, 30, 255])
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([40, 255, 255])
    
    # Create masks for each color
    mask_black = cv2.inRange(hsv_image, lower_black, upper_black)
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    
    # Combine masks
    combined_mask = cv2.bitwise_or(mask_black, mask_white)
    combined_mask = cv2.bitwise_or(combined_mask, mask_yellow)
    
    return cv2.bitwise_not(combined_mask)


def find_gray(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define thresholds for black, white, and yellow colors in HSV space
    lower_gray = np.array([0, 0, 90])
    upper_gray = np.array([180, 255, 150])
    
    # Create masks for each color
    mask_gray = cv2.inRange(hsv_image, lower_gray, upper_gray)

    # Combine masks
    return mask_gray

In [25]:
def getCountors(frame, gray = False, draw = True):
    if not gray:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        frame_gray = frame
    
    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    
    print(len(cnts))
    # cnt = [c[0] for c in cnts[:3]]
    cnt = [c[0] for c in cnts[:3]]
    # cnt = [c for c in cnts[:3]]
    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # Draw the center of the contour
        cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
        # contours = [c[0] for c in cnt]
        frame = cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)
        # for i in range(len(cnt)):
        #     print(i)
        #     x, y, w, h = cnt[i][4]
        #     frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # cnt = cnt[0]
    # l = tuple(cnt[cnt[:,:,0].argmin()][0])
    # r = tuple(cnt[cnt[:,:,0].argmax()][0])
    # t = tuple(cnt[cnt[:,:,1].argmin()][0])
    # b = tuple(cnt[cnt[:,:,1].argmax()][0])


    return frame, cnts

In [27]:
import cv2
import numpy as np

def find_colored_areas(image, color_range_red, color_range_green):
  """
  Finds red and green areas in an image using OpenCV.

  Args:
      image: The input image as a NumPy array.
      color_range_red: Lower and upper bounds for red color in HSV (Hue, Saturation, Value) format.
      color_range_green: Lower and upper bounds for green color in HSV format.

  Returns:
      A tuple containing two NumPy arrays:
          - red_mask: Binary mask where white pixels represent red areas.
          - green_mask: Binary mask where white pixels represent green areas.
  """
  # Convert image to HSV colorspace
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Create masks for red and green colors
  red_mask = cv2.inRange(hsv, color_range_red[0], color_range_red[1])
  green_mask = cv2.inRange(hsv, color_range_green[0], color_range_green[1])

  # Optional: Refine masks (noise reduction, contour smoothing)
  # ... (implement further processing as needed)

  return red_mask, green_mask

# Example usage (adjust color ranges as needed)
image = cv2.imread(path3)  # Replace with your image path

w, h = image.shape[1]//1, image.shape[0]//1

image = cv2.resize(image, (w, h))

color_range_red = (np.array([0, 75, 75], np.uint8), np.array([15, 255, 255], np.uint8))  # Lower red limit (adjust) 
color_range_green = (np.array([40, 50, 50], np.uint8), np.array([80, 255, 255], np.uint8))  # Lower green limit (adjust)

red_mask, green_mask = find_colored_areas(image, color_range_red, color_range_green)

# combined_mask = detect_black_white_yellow(image)
# kernel = np.ones((3,3),np.uint8)
# combined_mask_e = cv2.erode(combined_mask, kernel, iterations = 3)

# gray_mask = find_gray(image)


# Display results (optional)
kernel = np.ones((3,3),np.uint8)
red_mask_e = cv2.erode(red_mask, kernel, iterations = 3)
green_mask_e = cv2.erode(green_mask, kernel, iterations = 3)

# red_mask_e = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel, iterations = 4)
# green_mask_e = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel, iterations = 4)


img_cnts_green, cnts1 = getCountors(red_mask_e, gray = True)
img_cnts_red, cnts1 = getCountors(green_mask_e, gray = True)

cv2.imshow("Original Image", image)
# cv2.startWindowThread()
cv2.imshow("Red Mask M", img_cnts_green)
# cv2.startWindowThread()
cv2.imshow("Green Mask M", img_cnts_red)
# cv2.startWindowThread()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

33
2


-1

In [40]:
red_mask.shape

(1736, 2320)

In [14]:
np.unique(red_mask_e)

array([  0, 255], dtype=uint8)